### Bug 1: Rows don't append after minutes around 1:36:00 when there is only 1 line available for bet

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 19 01:36:28 2020

@author: TysonWu
"""

import pandas as pd
from datetime import datetime
from datetime import timedelta
from crawler import initialize
# odd_dict = dict(chl_line=[6.5,7.5], chl_hi = [1.0,2.0], chl_low=[3.0,4.0])
live_score_dict=dict(home=1, away=2)
event_id = '20200218TUE13'
game_starting_time = '456'

url ='https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=2d90153b-091e-4847-a806-d40f5bc32930'
driver=initialize(url)
def scrap_odds(driver, event_id):
    entry = "1"
    chl_line_list = []
    chl_hi_list = []
    chl_low_list = []
    while (True):
        try:
            chl_line_list.append(
                driver.find_element_by_id(event_id+'_CHL_LINE_'+entry).get_attribute(
                 'innerHTML').strip("[]"))
            chl_hi_list.append(
             driver.find_element_by_id(event_id+'_CHL_H_'+entry).get_attribute(
                 'innerHTML'))
            chl_low_list.append(
             driver.find_element_by_id(event_id+'_CHL_L_'+entry).get_attribute(
                 'innerHTML'))
            entry = str(int(entry) + 1)
        except:
            # return things when error
            return dict(chl_line=chl_line_list, chl_hi=chl_hi_list, chl_low=chl_low_list)

def make_odds_data(odd_dict, live_score_dict, event_id, game_starting_time):
    # add timestamp
    timestamp = datetime.strptime(datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S"), 
                                  "%Y-%m-%d %H:%M:%S")

    minutes = game_starting_time

    time_data = pd.DataFrame({"event_id": event_id,
                              "timestamp": timestamp,
                              "minutes": minutes},index=[0])

    # append live_score data
    odds = pd.concat([time_data, pd.DataFrame(live_score_dict, index=[0])], axis=1)

    # append corner hilow odds data
    odds = pd.concat([odds, pd.DataFrame(odd_dict)], axis=1).fillna(method="ffill")
    return odds
    


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/TysonWu/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]


In [4]:
odd_dict = scrap_odds(driver,event_id)
print(odd_dict)
odds = make_odds_data(odd_dict, live_score_dict, event_id, game_starting_time)

print(odds)

if not odds.chl_line.isnull().values.any():
    print('export to csv')

{'chl_line': ['10.5'], 'chl_hi': ['2.05'], 'chl_low': ['1.68']}
        event_id           timestamp minutes  home  away chl_line chl_hi  \
0  20200218TUE13 2020-02-19 02:37:33     456     1     2     10.5   2.05   

  chl_low  
0    1.68  
export to csv


In [3]:
odds.chl_line.isnull().values.any()

False

In [15]:
import os
path_file = event_id + ".csv"
if os.path.exists(path_file) == True:
    odds.to_csv(path_file, index=False, mode="a", header=False)
else:
    odds.to_csv(path_file, index=False, mode="w", header=True)

In [3]:
def f():
    entry = "1"
    chl_line_list = []
    chl_hi_list = []
    chl_low_list = []
    while (True):
        try:
            print(entry)
            chl_line_list.append(
                driver.find_element_by_id(event_id+'_CHL_LINE_'+entry).get_attribute(
                    'innerHTML').strip("[]"))
            chl_hi_list.append(
                driver.find_element_by_id(event_id+'_CHL_H_'+entry).get_attribute(
                    'innerHTML'))
            chl_low_list.append(
                driver.find_element_by_id(event_id+'_CHL_L_'+entry).get_attribute(
                    'innerHTML'))
            entry = str(int(entry) + 1)
        except:
            # return things when error
            print('enter except block')
            print(chl_line_list)
            print(chl_hi_list)
            print(chl_low_list)
            odd_dict = dict(chl_line=chl_line_list, chl_hi=chl_hi_list, chl_low=chl_low_list)
            return odd_dict
dictionary = f()
print(dictionary)

1
2
enter except block
['10.5']
['2.05']
['1.68']
{'chl_line': ['10.5'], 'chl_hi': ['2.05'], 'chl_low': ['1.68']}


### Developing 2: find total corner number

In [8]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from crawler import initialize

url ='https://bet.hkjc.com/football/odds/odds_inplay_all.aspx?lang=EN&tmatchid=2d90153b-091e-4847-a806-d40f5bc32930'
driver=initialize(url)



Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/TysonWu/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]


In [21]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.find_element_by_class_name('spTotalCorner').get_attribute('innerHTML'), 
                     'html.parser').text
soup

' '

In [23]:
if soup == '':
    print('empty')
if soup == ' ':
    print('one space')

one space
